In [19]:
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def train_logistic_regression(X_train, y_train, learning_rate, num_epochs):
    num_samples, num_features = X_train.shape
    num_classes = len(np.unique(y_train))


    W = np.zeros((num_features, num_classes))
    b = np.zeros(num_classes)

    for epoch in range(num_epochs):

        logits = np.dot(X_train, W) + b


        probabilities = softmax(logits)


        gradient = probabilities
        gradient[range(num_samples), y_train] -= 1
        gradient /= num_samples


        dW = np.dot(X_train.T, gradient)
        db = np.sum(gradient, axis=0)

        W -= learning_rate * dW
        b -= learning_rate * db

    return W, b


learning_rate = 0.01
num_epochs = 1000


W, b = train_logistic_regression(X_train, y_train, learning_rate, num_epochs)




In [20]:
def predict(X, W, b):
    logits = np.dot(X, W) + b
    probabilities = softmax(logits)
    return np.argmax(probabilities, axis=1)  


def split_data_kfold(data, k):
    return [data[i::k] for i in range(k)]

def get_train_val_test_splits(folds, fold_idx):
    val_data = folds[fold_idx]
    train_data = [data for i, data in enumerate(folds) if i != fold_idx]
    train_data = [item for sublist in train_data for item in sublist]
    
    num_train_samples = int(len(train_data) * 0.9)  
    train_data, inner_val_data = train_data[:num_train_samples], train_data[num_train_samples:]
    
    return train_data, inner_val_data, val_data

k = 5
folds = split_data_kfold(iris_data, k)

fold_accuracies = []

for i in range(k):
    train_data, inner_val_data, val_data = get_train_val_test_splits(folds, i)
    
    X_train = np.array([data[0] for data in train_data])
    y_train = np.array([data[1] for data in train_data])
    
    X_inner_val = np.array([data[0] for data in inner_val_data])
    y_inner_val = np.array([data[1] for data in inner_val_data])
    
    X_val = np.array([data[0] for data in val_data])
    y_val = np.array([data[1] for data in val_data])
    
    W, b = train_logistic_regression(X_train, y_train, learning_rate, num_epochs)
    
    y_inner_val_pred = predict(X_inner_val, W, b)
    
    accuracy = np.mean(y_inner_val_pred == y_inner_val)
    fold_accuracies.append(accuracy)
    
    print(f"Accuracy for fold {i+1}: {accuracy * 100:.2f}%")

best_fold = np.argmax(fold_accuracies) + 1
print(f"Best fold: {best_fold}")


Accuracy for fold 1: 100.00%
Accuracy for fold 2: 91.67%
Accuracy for fold 3: 100.00%
Accuracy for fold 4: 100.00%
Accuracy for fold 5: 100.00%
Best fold: 1
